In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alzheimers_Disease"
cohort = "GSE137202"

# Input paths
in_trait_dir = "../../input/GEO/Alzheimers_Disease"
in_cohort_dir = "../../input/GEO/Alzheimers_Disease/GSE137202"

# Output paths
out_data_file = "../../output/preprocess/Alzheimers_Disease/GSE137202.csv"
out_gene_data_file = "../../output/preprocess/Alzheimers_Disease/gene_data/GSE137202.csv"
out_clinical_data_file = "../../output/preprocess/Alzheimers_Disease/clinical_data/GSE137202.csv"
json_path = "../../output/preprocess/Alzheimers_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Reversion of Alzheimer’s disease signatures"
!Series_summary	"Chemical Checker (CC) is a resource that provides processed, harmonized and integrated bioactivity data on 800,000 small molecules. In the CC, bioactivity data are expressed in a vector format, which naturally extends the notion of chemical similarity between compounds to similarities between bioactivity signatures of different kinds. We experimentally validate that CC signatures can be used to reverse and mimic biological signatures of disease models and genetic perturbations."
!Series_summary	"We developed cellular models of Alzheimer’s disease (AD) by introducing familial AD (fAD) mutations into SH-SY5Y cells. Using CRISPR/Cas9-induced homology-directed repair, we obtained clones harboring the fAD PSEN1-M146V or the APP-V717F mutations. Three compounds (noscapine - 10 uM, palbociclib - 0.4 uM and AG-494 - 10uM) reverted fAD signatures. We confirmed that genes up-regulated in SH-SY5Y 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Yes, this dataset contains gene expression data from Affymetrix PrimeView arrays
is_gene_available = True

# 2.1 Data Availability
# The trait (Alzheimer's Disease) is represented by the genotype (row 1)
# The dataset contains genotypes: Wild Type vs fAD mutations (APP-V717F or PSEN1-M146V)
trait_row = 1

# Age and gender information are not available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert genotype information to binary trait (Alzheimer's Disease)."""
    if not value or ":" not in value:
        return None
    
    val = value.split(":", 1)[1].strip().lower()
    
    # Wild Type is control (0), the mutated genes (APP or PSEN1) represent Alzheimer's models (1)
    if "wild type" in val:
        return 0
    elif "mutated" in val:  # Either APP-V717F or PSEN1-M146V mutation
        return 1
    else:
        return None

def convert_age(value):
    """Convert age information - not available in this dataset."""
    return None

def convert_gender(value):
    """Convert gender information - not available in this dataset."""
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
# We assume clinical_data is already loaded from previous steps
# Use the sample_characteristics_dict to create a proper clinical data DataFrame
# Create a DataFrame with keys as columns and transpose to get the right shape
clinical_data = pd.DataFrame.from_dict(sample_characteristics_dict, orient='index').T

# Now we have clinical data in the right format for the geo_select_clinical_features function
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview the selected clinical features
print("Selected clinical features preview:")
preview_result = preview_df(selected_clinical_df)
print(preview_result)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file, index=False)


Selected clinical features preview:
{0: [nan], 1: [1.0], 2: [nan]}


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (like "11715100_at") appear to be microarray probe IDs from an Affymetrix platform
# They are not standard human gene symbols and will need to be mapped to proper gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns that store probe IDs and gene symbols
# From the preview of gene_annotation, I can see:
# - 'ID' column contains the probe identifiers (e.g., "11715100_at")
# - 'Gene Symbol' column contains the gene symbols (e.g., "HIST1H3G")

# 2. Get the gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print some information about the mapped gene data
print(f"Original probe count: {len(gene_annotation)}")
print(f"Mapped gene count: {len(gene_data)}")
print("First 10 genes after mapping:")
print(gene_data.index[:10])


Original probe count: 1530585
Mapped gene count: 19963
First 10 genes after mapping:
Index(['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAAS',
       'AACS', 'AACSP1'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (19758, 30)


Normalized gene data saved to ../../output/preprocess/Alzheimers_Disease/gene_data/GSE137202.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM4072905': [0.0], 'GSM4072906': [1.0], 'GSM4072907': [1.0], 'GSM4072908': [0.0], 'GSM4072909': [0.0], 'GSM4072910': [0.0], 'GSM4072911': [1.0], 'GSM4072912': [1.0], 'GSM4072913': [1.0], 'GSM4072914': [1.0], 'GSM4072915': [0.0], 'GSM4072916': [1.0], 'GSM4072917': [1.0], 'GSM4072918': [0.0], 'GSM4072919': [0.0], 'GSM4072920': [0.0], 'GSM4072921': [1.0], 'GSM4072922': [1.0], 'GSM4072923': [1.0], 'GSM4072924': [1.0], 'GSM4072925': [0.0], 'GSM4072926': [1.0], 'GSM4072927': [1.0], 'GSM4072928': [0.0], 'GSM4072929': [0.0], 'GSM4072930': [0.0], 'GSM4072931': [1.0], 'GSM4072932': [1.0], 'GSM4072933': [1.0], 'GSM4072934': [1.0]}
Clinical data saved to ../../output/preprocess/Alzheimers_Disease/clinical_data/GSE137202.csv
Linking clinical and genetic data...
Linked data shape: (30, 19759)
Handling missing 

Linked data shape after handling missing values: (30, 19759)
Checking for bias in trait distribution...
For the feature 'Alzheimers_Disease', the least common label is '0.0' with 12 occurrences. This represents 40.00% of the dataset.
The distribution of the feature 'Alzheimers_Disease' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Alzheimers_Disease/GSE137202.csv
